# Funkcje specjalne (_dunder methods_)

Wyrażenie:

`c = a + b`

zawiera trzy zmienne, operator przypisania oraz binarny operator sumy. W przypadku, gdy zmienne `a`,`b`,`c` są zmiennymi typu prostego jak `int`, `float`, Python zawiera operator sumy w bibliotece standardowej, który zostanie użyty. W przypadku, gdy `a`, `b` i `c` byłyby nowego typu, operatory należałoby zdefiniować w ramach tego typu. Język Python zawiera znacznie więcej możliwości pod względem syntaktyki. Przykładowo, aby klasa obsługiwała następujące operacje:

c = a + b

c()

Pod względem logicznym klasa musi zawierać co najmniej zdefiniowany operator sumy oraz funkcję `__call__`, której obsługa zostanie wywołania w momencie użycia nawiasów. W tym miejscu należy pamiętać o czytelności kodu. Zasada jest dokładnie taka sama jak w przypadku definiowania operatorów, własne funkcje specjalne i operatory muszą mieć logiczne uzasadnienie tzn. ich użycie nie powinno być mylące. W tym laboratorium zostanie omówiony sposób definiowania własnych operatorów oraz inne mechanizmy pokrewne.

## Funkcja `__call__`

Umożliwia tworzenie syntaktycznej konstrukcji podobnej do wywołania funkcji.

In [4]:
class Strip:

    def __init__(self, characters):
        self.characters = characters

    def __call__(self, string: str):
        return string.strip(self.characters)

strip = Strip('~')
print(strip('~test~'))

test


## Funkcja `__getattr__`, `__setattr__` i `__hasattr`

Funkcja `__getattr__` wywoływana jest jedynie, gdy klasa nie ma takiej zmiennej, więc jest to miejsce, w którym łatwo umieścić logikę związaną z brakującym atrybutem. Może również przydać się jako mechanizm tworzenia dynamicznych typów. Pokrewna funkcja `__getattribute__` wywoływana jest zawsze bez względu na to czy atrybut istnieje czy nie.

In [7]:
class Settings:
    sett_map = {
        'path': lambda self: self._path
    }

    def __init__(self, path):
        self._path = path

    def __getattr__(self, attr):
        return Settings.sett_map[attr](self)

setting = Settings('/tmp')
print(setting.path)

class Const:

    def __setattr__(self, name, value):
        if name in self.__dict__:
            raise ValueError("Cannot change a const attribute")

        self.__dict__[name] = value

    def __delattr__(self, name):
        if name in self.__dict__:
            raise ValueError("Cannot delete a const attribute")
        raise AttributeError("'{0}' object has no attribute '{1}'"
                             .format(self.__class__.__name__, name))

Const.path = '/tmp'
print(Const.path)

/tmp
/tmp


Powyższy przykład składa się z dwóch klas. Pierwsza z nich symuluje prosty sposób użycia atrybutów. Konfiguracja może zawierać różne opcje, co uzasadnia użycie dynamicznych zmiennych. W drugiej klasie można zauważyć sposób przechowywania wszystkich zmiennych w języku Python, tj. za pomocą słownika `__dict__`, który jest składnikiem każdej klasy. Funkcja `__hasattr__` zwraca wartość typu `bool`, jeśli istnieje dana zmienna w klasie.

## Funkcja `__get__`, `__set__` i `__delete__`

Użycie jest bardzo podobne jak w przypadku zestawu `__getattr__`, `__setattr__` i `__hasattr` z tą różnicą, że dotyczy implementacji właściwości, które zawierają _getter_ i _setter_. Dodatkowo inna jest sygnatura tych funkcji. Do obsługi należy zdefiniować dwie klasy. Jedna zawierająca zaimplementowane funkcje `__getattr__`, `__setattr__` oraz druga, która zawiera użycie pierwszej zdefiniowanej klasy.

In [12]:
import math

class Degree:
    def __init__(self, value=0.0):
        self.value = float(value)

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        self.value = float(value)

class Radian:
    def __get__(self, instance, owner):
        return math.radians(instance.degree)

    def __set__(self, instance, value):
        instance.degree = math.degrees(value)



class RadianDegreeCalculator:
    degree = Degree()
    radian = Radian()

calc = RadianDegreeCalculator()
calc.degree = 180.0
print(calc.radian)
calc.radian = math.pi
print(calc.degree)

3.141592653589793
180.0


Wzorcową jednostką są stopnie. Konwerter zmienia jedynie radiany na stopnie w przypadku przypisania wartości dla radianów i zmianę ze stopni na radiany w przypadku próby odczytu właściwości. Dodatkowo zarówno w klasie `Degree` oraz `Radian` w momencie wywołania `__get__` i `__set__` implementacja właściwości przekazuje bieżącą instancję klasy `RadianDegreeCalculator` (typ używa obu klas). Funkcja `__delete__` wywoływana jest w momencie wywołania właściwości ze słowem kluczowym `del`.

## Funkcja `__new__` i `__del__`

Funkcje są wywoływane w momencie kolejno, przed tworzeniem instancji klasy oraz w momencie zwolnienia obiektu (`del obj`). Funkcja `__new__` umożliwia kontrolowanie sposobu tworzenia klasy np. zmianę typu.

In [17]:
class Student:
    def __init__(self):
        print('Student')

class Teacher:
    def __init__(self):
        print('Teacher')

class Person:
    def __new__(cls, *args, **kwargs):
        if 'salary' in kwargs:
            return Teacher()
        else:
            return Student()

person = Person(salary=0)
person = Person(exams_count=math.nan)

Teacher
Student


{'test': 123}

## Słownik `__slots__`, `__dict__`, `__class__`

Temat słownika przechowującego wszystkie zmienne instancji klasy był już poruszany wielokrotnie. Zbiór `__slots__` jest relatywnie nowym pojęciem w języku Python. Umożliwia przekazanie środowisku uruchomieniowemu języka Python informacji o tym jakie zmienne klasa będzie posiadać, tak aby w momencie tworzenia instancji zarezerwować odpowiednią liczbę komórek pamięci. Pozwala to przyspieszyć działanie programu.

In [27]:
from time import time

class TestA:
    __slots__ = ['x', 'y', 'z']
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class TestB:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

iterations = 10000000

start_time = time()
for _ in range(iterations):
    TestA(1,2,3)

print(time() - start_time)

start_time = time()
for _ in range(iterations):
    TestB(1,2,3)

print(time() - start_time)

print(TestA(1,2,3).__class__)
print(TestA(1,2,3).__class__.__name__)


2.4929304122924805
2.8596513271331787
<class '__main__.TestA'>
TestA


Czym większa kolekcja tym różnica czasowa będzie większa. Zmienna wbudowana `__class__` przechowuje informacje o bieżącym typie danych.

## Reprezentacje klasy

W zależności od metody wyświetlania klasy istnieje możliwość obsługi dedykowanego dla niej formatowania. Technika przedefiniowana przynajmniej funkcji `__str__` przydaje się w trakcie debugowania. Wiele edytorów kodu języka Python używają tej informacji (okno _popup_ po najechaniu kursorem myszy na zmienną).

| Nazwa funkcji | Opis | Przykład użycia |
|-|-|-|
| `__format__(self, spec)` | Do obsługi funkcji formatującej `format`. W zmiennej `spec` mogą pojawić się różne opcje. | `format(obj, 'f')`
| `__bytes__(self)` | Przydatne do serializacji binarnej. | `bytes(obj)`
| `__str__(self)` | Należy zwrócić łańcuch znaków, który ma się pojawić w momencie konwersji klasy na łańcuch znaków. | `str(obj)` |
| `__repr__(self)` | Co ma zostać zwrócone po wywołaniu funkcji wbudowanej `repr`. | `repr(obj)` |

In [29]:
class Student:
    def __init__(self, name: str, surname: str):
        self.name = name
        self.surname = surname

    def __str__(self):
        return f'{self.name} {self.surname}'


print(Student('Jan', 'Kowalski'))
student = Student('Jan', 'Kowalski')
print(f'{student}')

Jan Kowalski
Jan Kowalski


Należy w tym miejscu podkreślić, że wiele funkcji wbudowanych bazuje na tych funkcjach, co umożliwia własnemu typowi na interakcje z nimi.

## Operatory porównania

Poniżej znajduje się tabela przedstawiająca najważniejsze operatory używane w instrukcjach warunkowych oraz funkcja `hash`.

| Nazwa funkcji | Opis |
|-|-|
`obj.__lt__(self.obj2)` | `x < y` |
`obj.__le__(self.obj2)` | `x <= y` |
`obj.__gt__(self.obj2)` | `x > y` |
`obj.__ge__(self.obj2)` | `x>= y` |
`obj.__eq__(self.obj2)` | `x == y` |
`obj.__ne__(self.obj2)` | `x != y` |
`obj.__bool__(self)` | `bool(x)` |
`obj.__hash__(self)` | `hash(x)` |
`obj.__add__` | `obj1 + obj2` |
`obj.__sub__` | `obj1 - obj2` |

Funkcja `hash` jest szczególnie istotna z punktu widzenia słowników i w momencie gdy klasa ta ma być kluczem w słowniku. Zostanie wywołana na instancji, przed próbą dodania nowego elementu do słownika. Na powyższej tabeli pokazano jedynie dwa operatory binarne. Ich lista jest znacznie bardziej obszerna i składa się ze wszystkich operatorów jakie są dostępne w języku Python.

In [38]:
class Test:
    def __init__(self, x: int):
        self.x = x

    def __bool__(self):
        return isinstance(self.x, int)

    def __lt__(self, other):
        return self.x < other.x

    def __hash__(self):
        return self.x

if Test(1):
    print('Yes')
else:
    print('No')

if Test(1.0):
    print('Yes')
else:
    print('No')

print(Test(1) < Test(2))

test_dict = { hash(Test(1)): 1, hash(Test(2)): 2 }
print(test_dict[hash(Test(1))])
print(test_dict[hash(Test(2))])

Yes
No
True
1
2


## Operacje na klasie jako kontenerze

Dodatkowe operatory, które nie występują w innych językach programowania jak `in` można również zdefiniować w klasie. Poniżej znajduje się przykład użycia klasy `Container`, która symuluje działanie kolekcji.

In [42]:
class Container:
    def __init__(self, n):
        self.n = n

    def __contains__(self, item):
        return -1 < item < self.n

    def __getitem__(self, position):
        return position

    def __setitem__(self, key, value):
        raise NotImplemented()

    def __iter__(self):
        for i in range(self.n):
            yield i

    def __len__(self):
        return self.n


container = Container(10)
print('in condition check')
print(1 in container) # __contains__
print(-1 in container) # __contains__
print('zero element')
print(container[0]) # __getitem__
print('Container as generator')
for i in container: # __iter__
    print(i)

print('len')
print(len(container)) # __len__

in condition check
True
False
zero element
0
Container as generator
0
1
2
3
4
5
6
7
8
9
len
10


## Funkcje `__enter__` i `__exit__`

Obie funkcje zostaną wywołane w momencie użycia konstrukcji `with` w języku Python. Funkcja `enter` zaraz po wejściu sterowania programu do __`with`__, a __`__exit__`__ w momencie opuszczenia bloku. Obie funkcje są zaimplementowane w funkcji standardowej `open`. W funkcji `__enter__` następuje wywołanie odpowiednich funkcji z bibliotek systemu operacyjnego do pobrania uchwytu do pliku, a w `__exit__` wykonanie sekwencji `flush` i `close`, ta ostatnia zwalnia uchwyt, co pozwala innemu programowi o pełny dostęp do pliku.

## Zadania do wykonania

### Zadanie 1
Napisz implementację drzewa wraz ze zdefiniowanymi operatorami jak:
- `len` ma zwracać wysokość drzewa,
- funkcja `count` ma zwracać liczbę wierzchołków,
- operator potęgowania  `__pow__` (`t**2`) ma wstawić losowe wartości, aż funkcja `count` nie będzie zwracać potęgi liczby pierwotnej np. przed `count(t)=3`, `t**2, count(t)=3**2`.
- iterator, który umożliwia przegląd wszystkich wierzchołków,
- sprawdzenie czy element znajduje się w drzewie za pomocą operatora `in` i `__getitem__`,
- zwrócić `False` jeśli drzewo jest puste oraz `True` w przeciwnym przypadku,
- dodawać elementy za pomocą `__setitem__` oraz `__lshift__` (operator `t << elem`),
- `str` ma zwracać narysowane drzewo,
- (opcjonalne) plus ma zwracać nowe drzewo złożone z dwóch,
- (opcjonalne) minus ma zwracać nowe drzewo będące różnicą.

###Zadanie 1

In [1]:
from drawtree import draw_bst

class Node():
    def __init__(self, data=None):
        self.data = data
        self.left_child = None
        self.right_child = None


class BST():
    def __init__(self):
        self.root = Node()
        self.nodes = 0
        self.list_of_nodes = list()

    def __setitem__(self, data):

        if self.root.data == None:
            self.root.data = data
        else:
            def add_to_node(data, node):
                if data < node.data:
                    if node.left_child == None:
                        node.left_child = Node(data)
                    else:
                        add_to_node(data, node.left_child)
                if data > node.data:
                    if node.right_child == None:
                        node.right_child = Node(data)
                    else:
                        add_to_node(data, node.right_child)

            add_to_node(data, self.root)
        self.nodes += 1
        self.list_of_nodes.append(data)

    def count(self):
        print(f'Liczba wierzchołków: {self.nodes}')

    def draw_tree(self):
        print('Wizualizacja drzewa: ')
        self.bst = draw_bst(self.list_of_nodes)


    def view_tree(self):
        result = ''

        def vlr(result, node):
            if node:
                if node.data:
                    result += (str(node.data) + '-')
                    result = vlr(result, node.left_child)
                    result = vlr(result, node.right_child)
            return result

        def lvr(result, node):
            if node:
                if node.data:
                    result = lvr(result, node.left_child)
                    result += (str(node.data) + '-')
                    result = lvr(result, node.right_child)
            return result

        def lrv(result, node):
            if node:
                if node.data:
                    result = lrv(result, node.left_child)
                    result = lrv(result, node.right_child)
                    result += (str(node.data) + '-')
            return result

        print(f'Preorder: {vlr(result, self.root)}')
        print(f'Inorder: {lvr(result, self.root)}')
        print(f'Postorder: {lrv(result, self.root)}')

    def is_empty(self):
        if self.root == None:
            return True
        else:
            return False

    def height(self, root):
        if root is None:
            return -1
        return max(self.height(root.left_child), self.height(root.right_child)) + 1

    def find(self, value):
        if self.root is None:
            return False
        return self.__getitem__(self.root, value)

    def __getitem__(self, node, data):
        if node is None:
            return False
        elif node.data == data:
            return True
        elif node.data > data:
            return self.__getitem__(node.left_child, data)
        else:
            return self.__getitem__(node.right_child, data)


tree = BST()
root = tree.root
tree.__setitem__(3)
tree.__setitem__(2)
tree.__setitem__(1)
tree.__setitem__(8)
tree.__setitem__(7)
tree.__setitem__(14)
tree.__setitem__(12)
tree.__setitem__(20)
print(f'Czy drzewo jest puste: {tree.is_empty()}')
print(f'Wysokość drzewa: {tree.height(root)}')
tree.count()
tree.draw_tree()
tree.view_tree()

print(tree.find(14))
print(tree.find(200))


Czy drzewo jest puste: False
Wysokość drzewa: 3
Liczba wierzchołków: 8
Wizualizacja drzewa: 
    3
   / \
  2   8
 /   / \
1   7  14
       / \
      /   \
     12   20
Preorder: 3-2-1-8-7-14-12-20-
Inorder: 1-2-3-7-8-12-14-20-
Postorder: 1-2-7-12-20-14-8-3-
True
False
